In [60]:
import numpy as np
from scipy.linalg import eigh

alpha = np.array([0.298073, 1.242567, 5.782948, 38.474970], dtype="float64")
c = np.array([1,2,3,4], dtype="float64")

In [131]:
Q = np.zeros((4,4,4,4))
S = np.zeros((4,4))
A = np.zeros((4,4))
T = np.zeros((4,4))
h = np.zeros((4,4))
F = np.zeros((4,4))

# build Q
for p in range(4):
    for q in range(4):
        for r in range(4):
            for s in range(4):
                numQ = ( 2 * ((np.pi)**(5/2)) ) 
                denQ = ( alpha[p] + alpha [q] ) * ( alpha[r] + alpha[s] ) \
                * np.sqrt( alpha[p] + alpha [q] + alpha[r] + alpha[s])
                Q[p][r][q][s] = numQ/denQ
                
# build S
for p in range(4):
    for q in range(4):
        S[q][p] = ( np.pi / ( alpha[p] + alpha[q] ) ) ** (3/2)
            
# build T
for p in range(4):
    for q in range(4):
        T[p][q] = 3 * (( ( alpha[p]*alpha[q] )*( np.pi**(3/2) ) ) \
                        / ( ( alpha[p] + alpha[q] )**(5/2) ))
        
# build A
for p in range(4):
    for q in range(4):
        A[p][q] = - ( (2 * np.pi) / ( alpha[p] + alpha[q] ) )
        
# build h
for p in range(4):
    for q in range(4):
        h[p][q] = 2*A[p][q] + T[p][q]

In [119]:
# C normalization
def c_norm(c,S):
    NormSquared = c@S@c
    c = c/np.sqrt(NormSquared)
    return c

c = c_norm(c,S)

In [123]:
def build_F(h,Q,c):
    F1 = np.zeros((4,4))
    for r in range(4):
        for s in range(4):
            F1[p][q] = Q[p][r][q][s]*c[r]*c[s]
    
    return (h + F1)

def build_Eg(c,h,Q):
    
    Eg1 = 0
    Eg2 = 0
    
    for p in range(4):
        for r in range(4):
            for q in range(4):
                for s in range(4):
                    Eg1 = Eg1 + Q[p][r][q][s]*c[p]*c[q]*c[r]*c[s]
                    
    for p in range(4):
        for q in range(4):
            Eg2 = Eg2 + c[p]*c[q]*h[p][q]
            
    return Eg1 + 2 * Eg2    

In [134]:
E = 0.
new_E = 1000.

while( abs(E-new_E) > 1e-7 ):
    
    F=build_F(h,Q,c)
    E = new_E
    
    # Solve the generalized eigenvalue problem Fc = E'Sc
    eigenvalues, eigenvectors = \
    eigh(F, b=S, eigvals_only=False, overwrite_a=True, overwrite_b=True)
    sort_indices = np.argsort(eigenvalues)
    eigenvalues = eigenvalues[sort_indices]
    eigenvectors = eigenvectors[:, sort_indices]
    
    c = c_norm(eigenvectors[0], S)
    new_E = build_Eg(c,h,Q)


[ 0.08704177  0.38509941  0.17304437 -0.05073576]
[ 0.08704177  0.38509941  0.17304437 -0.05073576]


In [135]:
result = build_Eg(c,h,Q)
print(result)

-2.690983846221795
